In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv("NY property data.csv")

In [0]:
ny = data.drop(["EASEMENT","OWNER","EXT","EXLAND","EXTOT","EXTOT", "EXCD1", "STADDR", "EXMPTCL",
        "AVLAND2", "AVTOT2", "EXLAND2", "EXTOT2",
       "EXCD2", "PERIOD", "YEAR", "VALTYPE"], axis = 1)

In [0]:
ny["ZIP"].fillna(method ='ffill', inplace = True)

In [0]:
ny.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH
0,1,1000010101,1,1,101,P7,4,500,1046,NaN,21400000.0,4225500.0,9630000.0,10004.0,0,0
1,2,1000010201,1,1,201,Z9,4,27,0,NaN,193800000.0,14310000.0,87210000.0,10004.0,0,0
2,3,1000020001,1,2,1,Y7,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004.0,709,564
3,4,1000020023,1,2,23,T2,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004.0,85,551
4,5,1000030001,1,3,1,Q1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004.0,89,57


In [0]:
ny.shape

(1070994, 16)

## Fill in BLDFRONT

In [0]:
ny_nonnull = ny[ny["BLDFRONT"] != 0]

In [0]:
# Level 1 - groupby zip, taxclass, bldgcl
## Extract null

ny_bdft_null = ny[ny.BLDFRONT == 0]

In [0]:
ny_bdft_null.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH
0,1,1000010101,1,1,101,P7,4,500,1046,NaN,21400000.0,4225500.0,9630000.0,10004.0,0,0
1,2,1000010201,1,1,201,Z9,4,27,0,NaN,193800000.0,14310000.0,87210000.0,10004.0,0,0
5,6,1000030002,1,3,2,Q1,4,496,76,NaN,10600000.0,4770000.0,4770000.0,10004.0,0,0
8,9,1000041001,1,4,1001,R5,4,0,0,50.0,3944762.0,636093.0,1775143.0,10004.0,0,0
9,10,1000041002,1,4,1002,R5,4,0,0,50.0,5700930.0,919276.0,2565419.0,10004.0,0,0


In [0]:
ny_bdft_null.shape

(228815, 16)

In [0]:
count1 = ny.groupby(["ZIP","TAXCLASS","BLDGCL"]).count().reset_index()
count1 = count1[count1["BLDFRONT"]>5]
count1.head()

,ZIP,TAXCLASS,BLDGCL,RECORD,BBLE,B,BLOCK,LOT,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,BLDFRONT,BLDDEPTH
8,10001.0,2,C4,27,27,27,27,27,27,27,27,27,27,27,27,27
9,10001.0,2,C5,17,17,17,17,17,17,17,17,17,17,17,17,17
10,10001.0,2,C6,6,6,6,6,6,6,6,6,6,6,6,6,6
11,10001.0,2,C7,15,15,15,15,15,15,15,15,15,15,15,15,15
12,10001.0,2,D0,17,17,17,17,17,17,17,17,17,17,17,17,17


In [0]:
count1 = count1.drop(["BBLE","B","BLOCK","RECORD","LOT","LTFRONT","LTDEPTH","STORIES","FULLVAL",
             "AVLAND","AVTOT","BLDFRONT","BLDDEPTH"], axis = 1)

In [0]:
count1.head()

,ZIP,TAXCLASS,BLDGCL
8,10001.0,2,C4
9,10001.0,2,C5
10,10001.0,2,C6
11,10001.0,2,C7
12,10001.0,2,D0


In [0]:
count1.shape

(7669, 3)

In [0]:
blft_median1 = pd.DataFrame(ny_nonnull.groupby(["ZIP","TAXCLASS","BLDGCL"])["BLDFRONT"].median()).reset_index()

In [0]:
blft_median1.head()

,ZIP,TAXCLASS,BLDGCL,BLDFRONT
0,10001.0,1,A4,16.0
1,10001.0,1,A5,18.0
2,10001.0,1,A9,16.5
3,10001.0,1,B9,17.0
4,10001.0,1,C0,20.0


In [0]:
blft_median1.shape

(14350, 4)

In [0]:
blft_lvl1 = count1.merge(blft_median1, how = "left", left_on = ["ZIP", "TAXCLASS", "BLDGCL"], 
                               right_on =["ZIP", "TAXCLASS", "BLDGCL"] )

In [0]:
blft_lvl1.head()

,ZIP,TAXCLASS,BLDGCL,BLDFRONT
0,10001.0,2,C4,25.0
1,10001.0,2,C5,22.0
2,10001.0,2,C6,34.5
3,10001.0,2,C7,25.0
4,10001.0,2,D0,53.0


In [0]:
blft_lvl1 = blft_lvl1[blft_lvl1["BLDFRONT"]>0]
blft_lvl1.head()

,ZIP,TAXCLASS,BLDGCL,BLDFRONT
0,10001.0,2,C4,25.0
1,10001.0,2,C5,22.0
2,10001.0,2,C6,34.5
3,10001.0,2,C7,25.0
4,10001.0,2,D0,53.0


In [0]:
blft_lvl1.shape

(6762, 4)

In [0]:
ny_bdft_1 = ny_bdft_null.merge(blft_lvl1, how = "left", left_on = ["ZIP", "TAXCLASS","BLDGCL"], 
                               right_on =["ZIP", "TAXCLASS", "BLDGCL"])

In [0]:
ny_bdft_1.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT_x,BLDDEPTH,BLDFRONT_y
0,1,1000010101,1,1,101,P7,4,500,1046,NaN,21400000.0,4225500.0,9630000.0,10004.0,0,0,NaN
1,2,1000010201,1,1,201,Z9,4,27,0,NaN,193800000.0,14310000.0,87210000.0,10004.0,0,0,NaN
2,6,1000030002,1,3,2,Q1,4,496,76,NaN,10600000.0,4770000.0,4770000.0,10004.0,0,0,NaN
3,9,1000041001,1,4,1001,R5,4,0,0,50.0,3944762.0,636093.0,1775143.0,10004.0,0,0,382.0
4,10,1000041002,1,4,1002,R5,4,0,0,50.0,5700930.0,919276.0,2565419.0,10004.0,0,0,382.0


In [0]:
ny_bdft_1 = ny_bdft_1.drop("BLDFRONT_x", axis = 1)

In [0]:
ny_bdft_1f = ny_bdft_1[ny_bdft_1["BLDFRONT_y"]>0]

In [0]:
ny_bdft_1f.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDDEPTH,BLDFRONT_y
3,9,1000041001,1,4,1001,R5,4,0,0,50.0,3944762.0,636093.0,1775143.0,10004.0,0,382.0
4,10,1000041002,1,4,1002,R5,4,0,0,50.0,5700930.0,919276.0,2565419.0,10004.0,0,382.0
5,11,1000041003,1,4,1003,R5,4,0,0,50.0,10600000.0,967500.0,4770000.0,10004.0,0,382.0
6,12,1000041004,1,4,1004,R5,4,0,0,50.0,1011928.0,163174.0,455368.0,10004.0,0,382.0
7,13,1000041005,1,4,1005,R5,4,0,0,50.0,2318026.0,373783.0,1043112.0,10004.0,0,382.0


In [0]:
ny_bdft_1f.shape

(153178, 16)

In [0]:
## Level 2 - groupby zip, taxclass
ny_bdft_null2 = ny_bdft_1[ny_bdft_1["BLDFRONT_y"].isnull()]

In [0]:
ny_bdft_null2.shape

(75637, 16)

In [0]:
count2 = ny.groupby(["ZIP","TAXCLASS"]).count().reset_index()
count2 = count2[count2["RECORD"]>5]
count2.head()

,ZIP,TAXCLASS,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,BLDFRONT,BLDDEPTH
0,10001.0,1,19,19,19,19,19,19,19,19,19,19,19,19,19,19
1,10001.0,2,1089,1089,1089,1089,1089,1089,1089,1089,1086,1089,1089,1089,1089,1089
2,10001.0,2A,46,46,46,46,46,46,46,46,46,46,46,46,46,46
3,10001.0,2B,38,38,38,38,38,38,38,38,38,38,38,38,38,38
4,10001.0,2C,47,47,47,47,47,47,47,47,47,47,47,47,47,47


In [0]:
count2 = count2.drop(["BBLE","B","BLOCK","RECORD","LOT","BLDGCL","LTFRONT","LTDEPTH","STORIES","FULLVAL",
             "AVLAND","AVTOT","BLDFRONT","BLDDEPTH"], axis = 1)

In [0]:
count2.head()

,ZIP,TAXCLASS
0,10001.0,1
1,10001.0,2
2,10001.0,2A
3,10001.0,2B
4,10001.0,2C


In [0]:
blft_median2 = pd.DataFrame(ny_nonnull.groupby(["ZIP","TAXCLASS"])["BLDFRONT"].median()).reset_index()

In [0]:
blft_median2.head()

,ZIP,TAXCLASS,BLDFRONT
0,10001.0,1,19.0
1,10001.0,2,53.0
2,10001.0,2A,20.0
3,10001.0,2B,25.0
4,10001.0,2C,25.0


In [0]:
blft_lvl2 = count2.merge(blft_median2, how = "left", left_on = ["ZIP", "TAXCLASS"], 
                               right_on =["ZIP", "TAXCLASS"] )

In [0]:
blft_lvl2.head()

,ZIP,TAXCLASS,BLDFRONT
0,10001.0,1,19.0
1,10001.0,2,53.0
2,10001.0,2A,20.0
3,10001.0,2B,25.0
4,10001.0,2C,25.0


In [0]:
blft_lvl2 = blft_lvl2[blft_lvl2["BLDFRONT"]>0]
blft_lvl2.head()

,ZIP,TAXCLASS,BLDFRONT
0,10001.0,1,19.0
1,10001.0,2,53.0
2,10001.0,2A,20.0
3,10001.0,2B,25.0
4,10001.0,2C,25.0


In [0]:
ny_bdft_2 = ny_bdft_null2.merge(blft_lvl2, how = "left", left_on = ["ZIP", "TAXCLASS"], 
                               right_on =["ZIP", "TAXCLASS"])

In [0]:
ny_bdft_2.shape

(75637, 17)

In [0]:
ny_bdft_2.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDDEPTH,BLDFRONT_y,BLDFRONT
0,1,1000010101,1,1,101,P7,4,500,1046,NaN,21400000.0,4225500.0,9630000.0,10004.0,0,NaN,152.0
1,2,1000010201,1,1,201,Z9,4,27,0,NaN,193800000.0,14310000.0,87210000.0,10004.0,0,NaN,152.0
2,6,1000030002,1,3,2,Q1,4,496,76,NaN,10600000.0,4770000.0,4770000.0,10004.0,0,NaN,152.0
3,71,1000060001,1,6,1,Q9,4,234,349,NaN,50040000.0,22518000.0,22518000.0,10004.0,0,NaN,152.0
4,72,1000060001E,1,6,1,Z7,4,191,349,NaN,135000.0,24750.0,60750.0,10004.0,0,NaN,152.0


In [0]:
ny_bdft_2 = ny_bdft_2.drop("BLDFRONT_y", axis = 1)

In [0]:
ny_bdft_2.rename(columns = {'BLDFRONT':'BLDFRONT_y'}, inplace = True) 

In [0]:
ny_bdft_2f = ny_bdft_2[ny_bdft_2["BLDFRONT_y"]>0]

In [0]:
ny_bdft_2f.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDDEPTH,BLDFRONT_y
0,1,1000010101,1,1,101,P7,4,500,1046,NaN,21400000.0,4225500.0,9630000.0,10004.0,0,152.0
1,2,1000010201,1,1,201,Z9,4,27,0,NaN,193800000.0,14310000.0,87210000.0,10004.0,0,152.0
2,6,1000030002,1,3,2,Q1,4,496,76,NaN,10600000.0,4770000.0,4770000.0,10004.0,0,152.0
3,71,1000060001,1,6,1,Q9,4,234,349,NaN,50040000.0,22518000.0,22518000.0,10004.0,0,152.0
4,72,1000060001E,1,6,1,Z7,4,191,349,NaN,135000.0,24750.0,60750.0,10004.0,0,152.0


In [0]:
ny_bdft_2f.shape

(44411, 16)

In [0]:
## Level 3 - groupby zip
ny_bdft_null3 = ny_bdft_2[ny_bdft_2["BLDFRONT_y"].isnull()]

In [0]:
ny_bdft_null3.shape

(31226, 16)

In [0]:
blft_median3 = pd.DataFrame(ny_nonnull.groupby(["ZIP"])["BLDFRONT"].median()).reset_index()

In [0]:
blft_median3.head()

,ZIP,BLDFRONT
0,10001.0,42.0
1,10002.0,25.0
2,10003.0,26.0
3,10004.0,132.0
4,10005.0,151.0


In [0]:
ny_bdft_3 = ny_bdft_null3.merge(blft_median3, how = "left", left_on = ["ZIP"], 
                               right_on =["ZIP"])

In [0]:
ny_bdft_3.shape

(31226, 17)

In [0]:
ny_bdft_3.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDDEPTH,BLDFRONT_y,BLDFRONT
0,89,1000080044,1,8,44,U4,3,111,138,NaN,0.0,0.0,0.0,10004.0,0,NaN,132.0
1,127,1000130027E,1,13,27,U7,3,0,0,NaN,0.0,0.0,0.0,10004.0,0,NaN,132.0
2,6309,1000411101,1,41,1101,R8,2C,0,0,5.0,2482000.0,15046.0,206481.0,10005.0,0,NaN,151.0
3,6310,1000411102,1,41,1102,R1,2C,0,0,5.0,607778.0,9568.0,169787.0,10005.0,0,NaN,151.0
4,6311,1000411103,1,41,1103,R1,2C,0,0,NaN,652631.0,10131.0,179772.0,10005.0,0,NaN,151.0


In [0]:
ny_bdft_3 = ny_bdft_3.drop("BLDFRONT_y", axis = 1)

In [0]:
ny_bdft_3.rename(columns = {'BLDFRONT':'BLDFRONT_y'}, inplace = True) 

In [0]:
ny_bdft_3.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDDEPTH,BLDFRONT_y
0,89,1000080044,1,8,44,U4,3,111,138,NaN,0.0,0.0,0.0,10004.0,0,132.0
1,127,1000130027E,1,13,27,U7,3,0,0,NaN,0.0,0.0,0.0,10004.0,0,132.0
2,6309,1000411101,1,41,1101,R8,2C,0,0,5.0,2482000.0,15046.0,206481.0,10005.0,0,151.0
3,6310,1000411102,1,41,1102,R1,2C,0,0,5.0,607778.0,9568.0,169787.0,10005.0,0,151.0
4,6311,1000411103,1,41,1103,R1,2C,0,0,NaN,652631.0,10131.0,179772.0,10005.0,0,151.0


In [0]:
ny_bdft_3f = ny_bdft_3[ny_bdft_3["BLDFRONT_y"] != 0]

In [0]:
ny_bdft_3f.shape

(31226, 16)

In [0]:
ny_bdft_3[ny_bdft_3["BLDFRONT_y"] == 0]

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDDEPTH,BLDFRONT_y


In [0]:
## Combine
ny_bdft_f = pd.concat([ny_bdft_1f,ny_bdft_2f,ny_bdft_3f])

In [0]:
ny_bdft_f.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDDEPTH,BLDFRONT_y
3,9,1000041001,1,4,1001,R5,4,0,0,50.0,3944762.0,636093.0,1775143.0,10004.0,0,382.0
4,10,1000041002,1,4,1002,R5,4,0,0,50.0,5700930.0,919276.0,2565419.0,10004.0,0,382.0
5,11,1000041003,1,4,1003,R5,4,0,0,50.0,10600000.0,967500.0,4770000.0,10004.0,0,382.0
6,12,1000041004,1,4,1004,R5,4,0,0,50.0,1011928.0,163174.0,455368.0,10004.0,0,382.0
7,13,1000041005,1,4,1005,R5,4,0,0,50.0,2318026.0,373783.0,1043112.0,10004.0,0,382.0


In [0]:
ny_bdft_f.rename(columns = {'BLDFRONT_y':'BLDFRONT'}, inplace = True) 

In [0]:
cols = list(ny_nonnull.columns.values)

In [0]:
cols

['RECORD',
 'BBLE',
 'B',
 'BLOCK',
 'LOT',
 'BLDGCL',
 'TAXCLASS',
 'LTFRONT',
 'LTDEPTH',
 'STORIES',
 'FULLVAL',
 'AVLAND',
 'AVTOT',
 'ZIP',
 'BLDFRONT',
 'BLDDEPTH']

In [0]:
ny_bdft_f = ny_bdft_f[['RECORD',
 'BBLE',
 'B',
 'BLOCK',
 'LOT',
 'BLDGCL',
 'TAXCLASS',
 'LTFRONT',
 'LTDEPTH',
 'STORIES',
 'FULLVAL',
 'AVLAND',
 'AVTOT',
 'ZIP',
 'BLDFRONT',
 'BLDDEPTH']]

In [0]:
ny_bdft_f.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH
3,9,1000041001,1,4,1001,R5,4,0,0,50.0,3944762.0,636093.0,1775143.0,10004.0,382.0,0
4,10,1000041002,1,4,1002,R5,4,0,0,50.0,5700930.0,919276.0,2565419.0,10004.0,382.0,0
5,11,1000041003,1,4,1003,R5,4,0,0,50.0,10600000.0,967500.0,4770000.0,10004.0,382.0,0
6,12,1000041004,1,4,1004,R5,4,0,0,50.0,1011928.0,163174.0,455368.0,10004.0,382.0,0
7,13,1000041005,1,4,1005,R5,4,0,0,50.0,2318026.0,373783.0,1043112.0,10004.0,382.0,0


In [0]:
ny_BDFRONT = pd.concat([ny_nonnull,ny_bdft_f])

In [0]:
## Check
ny_BDFRONT.shape

(1070994, 16)

In [0]:
ny_BDFRONT[ny_BDFRONT["BLDFRONT"] == 0]

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH


In [0]:
ny_BDFRONT.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH
2,3,1000020001,1,2,1,Y7,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004.0,709.0,564
3,4,1000020023,1,2,23,T2,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004.0,85.0,551
4,5,1000030001,1,3,1,Q1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004.0,89.0,57
6,7,1000030003,1,3,3,Q1,4,180,370,1.0,8489997.0,3820499.0,3820499.0,10004.0,16.0,19
7,8,1000030010,1,3,10,Z9,4,362,177,3.0,28100000.0,11070000.0,12645000.0,10004.0,37.0,227


In [0]:
ny = ny_BDFRONT.sort_values("RECORD", ascending = True)

In [0]:
ny.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH
0,1,1000010101,1,1,101,P7,4,500,1046,NaN,21400000.0,4225500.0,9630000.0,10004.0,152.0,0
1,2,1000010201,1,1,201,Z9,4,27,0,NaN,193800000.0,14310000.0,87210000.0,10004.0,152.0,0
2,3,1000020001,1,2,1,Y7,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004.0,709.0,564
3,4,1000020023,1,2,23,T2,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004.0,85.0,551
4,5,1000030001,1,3,1,Q1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004.0,89.0,57


## Fill in BLDDEPTH

In [0]:
ny_nonnull = ny[ny["BLDDEPTH"] != 0]

In [0]:
# Level 1 - groupby zip, taxclass, bldgcl
## Extract null

ny_bddp_null = ny[ny.BLDDEPTH == 0]

In [0]:
ny_bddp_null.shape

(228853, 16)

In [0]:
count1 = ny.groupby(["ZIP","TAXCLASS","BLDGCL"]).count().reset_index()
count1 = count1[count1["BLDDEPTH"]>5]
count1.head()

,ZIP,TAXCLASS,BLDGCL,RECORD,BBLE,B,BLOCK,LOT,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,BLDFRONT,BLDDEPTH
8,10001.0,2,C4,27,27,27,27,27,27,27,27,27,27,27,27,27
9,10001.0,2,C5,17,17,17,17,17,17,17,17,17,17,17,17,17
10,10001.0,2,C6,6,6,6,6,6,6,6,6,6,6,6,6,6
11,10001.0,2,C7,15,15,15,15,15,15,15,15,15,15,15,15,15
12,10001.0,2,D0,17,17,17,17,17,17,17,17,17,17,17,17,17


In [0]:
count1 = count1.drop(["BBLE","B","BLOCK","RECORD","LOT","LTFRONT","LTDEPTH","STORIES","FULLVAL",
             "AVLAND","AVTOT","BLDFRONT","BLDDEPTH"], axis = 1)

In [0]:
bldp_median1 = pd.DataFrame(ny_nonnull.groupby(["ZIP","TAXCLASS","BLDGCL"])["BLDDEPTH"].median()).reset_index()

In [0]:
bldp_lvl1 = count1.merge(bldp_median1, how = "left", left_on = ["ZIP", "TAXCLASS", "BLDGCL"], 
                               right_on =["ZIP", "TAXCLASS", "BLDGCL"] )

In [0]:
bldp_lvl1 = bldp_lvl1[bldp_lvl1["BLDDEPTH"]>0]
bldp_lvl1.head()

,ZIP,TAXCLASS,BLDGCL,BLDDEPTH
0,10001.0,2,C4,85.0
1,10001.0,2,C5,54.0
2,10001.0,2,C6,81.5
3,10001.0,2,C7,80.0
4,10001.0,2,D0,96.0


In [0]:
ny_bddp_1 = ny_bddp_null.merge(bldp_lvl1, how = "left", left_on = ["ZIP", "TAXCLASS","BLDGCL"], 
                               right_on =["ZIP", "TAXCLASS", "BLDGCL"])

In [0]:
ny_bddp_1 = ny_bddp_1.drop("BLDDEPTH_x", axis = 1)

In [0]:
ny_bddp_1f = ny_bddp_1[ny_bddp_1["BLDDEPTH_y"]>0]

In [0]:
ny_bddp_1f.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH_y
3,9,1000041001,1,4,1001,R5,4,0,0,50.0,3944762.0,636093.0,1775143.0,10004.0,382.0,318.0
4,10,1000041002,1,4,1002,R5,4,0,0,50.0,5700930.0,919276.0,2565419.0,10004.0,382.0,318.0
5,11,1000041003,1,4,1003,R5,4,0,0,50.0,10600000.0,967500.0,4770000.0,10004.0,382.0,318.0
6,12,1000041004,1,4,1004,R5,4,0,0,50.0,1011928.0,163174.0,455368.0,10004.0,382.0,318.0
7,13,1000041005,1,4,1005,R5,4,0,0,50.0,2318026.0,373783.0,1043112.0,10004.0,382.0,318.0


In [0]:
ny_bddp_1f.shape

(153212, 16)

In [0]:
## Level 2 - groupby zip, taxclass
ny_bddp_null2 = ny_bddp_1[ny_bddp_1["BLDDEPTH_y"].isnull()]

In [0]:
count2 = ny.groupby(["ZIP","TAXCLASS"]).count().reset_index()
count2 = count2[count2["RECORD"]>5]
count2.head()

,ZIP,TAXCLASS,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,BLDFRONT,BLDDEPTH
0,10001.0,1,19,19,19,19,19,19,19,19,19,19,19,19,19,19
1,10001.0,2,1089,1089,1089,1089,1089,1089,1089,1089,1086,1089,1089,1089,1089,1089
2,10001.0,2A,46,46,46,46,46,46,46,46,46,46,46,46,46,46
3,10001.0,2B,38,38,38,38,38,38,38,38,38,38,38,38,38,38
4,10001.0,2C,47,47,47,47,47,47,47,47,47,47,47,47,47,47


In [0]:
count2 = count2.drop(["BBLE","B","BLOCK","RECORD","LOT","BLDGCL","LTFRONT","LTDEPTH","STORIES","FULLVAL",
             "AVLAND","AVTOT","BLDFRONT","BLDDEPTH"], axis = 1)

In [0]:
bldp_median2 = pd.DataFrame(ny_nonnull.groupby(["ZIP","TAXCLASS"])["BLDDEPTH"].median()).reset_index()

In [0]:
bldp_lvl2 = count2.merge(bldp_median2, how = "left", left_on = ["ZIP", "TAXCLASS"], 
                               right_on =["ZIP", "TAXCLASS"] )

In [0]:
bldp_lvl2 = bldp_lvl2[bldp_lvl2["BLDDEPTH"]>0]
bldp_lvl2.head()

,ZIP,TAXCLASS,BLDDEPTH
0,10001.0,1,46.0
1,10001.0,2,98.0
2,10001.0,2A,59.5
3,10001.0,2B,64.0
4,10001.0,2C,92.0


In [0]:
ny_bddp_2 = ny_bddp_null2.merge(bldp_lvl2, how = "left", left_on = ["ZIP", "TAXCLASS"], 
                               right_on =["ZIP", "TAXCLASS"])

In [0]:
ny_bddp_2 = ny_bddp_2.drop("BLDDEPTH_y", axis = 1)

In [0]:
ny_bddp_2.rename(columns = {'BLDDEPTH':'BLDDEPTH_y'}, inplace = True) 

In [0]:
ny_bddp_2f = ny_bddp_2[ny_bddp_2["BLDDEPTH_y"]>0]

In [0]:
ny_bddp_2f.shape

(44513, 16)

In [0]:
## Level 3 - groupby zip
ny_bddp_null3 = ny_bddp_2[ny_bddp_2["BLDDEPTH_y"].isnull()]

In [0]:
bldp_median3 = pd.DataFrame(ny_nonnull.groupby(["ZIP"])["BLDDEPTH"].median()).reset_index()

In [0]:
ny_bddp_3 = ny_bddp_null3.merge(bldp_median3, how = "left", left_on = ["ZIP"], 
                               right_on =["ZIP"])

In [0]:
ny_bddp_3 = ny_bddp_3.drop("BLDDEPTH_y", axis = 1)

In [0]:
ny_bddp_3.rename(columns = {'BLDDEPTH':'BLDDEPTH_y'}, inplace = True) 

In [0]:
ny_bddp_3f = ny_bddp_3[ny_bddp_3["BLDDEPTH_y"] != 0]

In [0]:
ny_bddp_3f.shape

(31128, 16)

In [0]:
## Combine
ny_bddp_f = pd.concat([ny_bddp_1f,ny_bddp_2f,ny_bddp_3f])

In [0]:
ny_bddp_f.rename(columns = {'BLDDEPTH_y':'BLDDEPTH'}, inplace = True) 

In [0]:
ny_bddp_f.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH
3,9,1000041001,1,4,1001,R5,4,0,0,50.0,3944762.0,636093.0,1775143.0,10004.0,382.0,318.0
4,10,1000041002,1,4,1002,R5,4,0,0,50.0,5700930.0,919276.0,2565419.0,10004.0,382.0,318.0
5,11,1000041003,1,4,1003,R5,4,0,0,50.0,10600000.0,967500.0,4770000.0,10004.0,382.0,318.0
6,12,1000041004,1,4,1004,R5,4,0,0,50.0,1011928.0,163174.0,455368.0,10004.0,382.0,318.0
7,13,1000041005,1,4,1005,R5,4,0,0,50.0,2318026.0,373783.0,1043112.0,10004.0,382.0,318.0


In [0]:
ny_final = pd.concat([ny_nonnull,ny_bddp_f])

In [0]:
ny = ny_final.sort_values("RECORD", ascending = True)

In [0]:
## Check
ny.shape

(1070994, 16)

In [0]:
ny[ny.BLDDEPTH == 0]

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH


In [0]:
ny.head()

,RECORD,BBLE,B,BLOCK,LOT,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH
0,1,1000010101,1,1,101,P7,4,500,1046,NaN,21400000.0,4225500.0,9630000.0,10004.0,152.0,207.0
1,2,1000010201,1,1,201,Z9,4,27,0,NaN,193800000.0,14310000.0,87210000.0,10004.0,152.0,207.0
2,3,1000020001,1,2,1,Y7,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004.0,709.0,564.0
3,4,1000020023,1,2,23,T2,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004.0,85.0,551.0
4,5,1000030001,1,3,1,Q1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004.0,89.0,57.0


In [0]:
ny_bldfront_blddepth = ny.drop(["BBLE","B","BLOCK","LOT","BLDGCL","TAXCLASS","LTFRONT","LTDEPTH","STORIES","FULLVAL",
             "AVLAND","AVTOT","ZIP"], axis = 1)

In [0]:
ny_bldfront_blddepth.head()

,RECORD,BLDFRONT,BLDDEPTH
0,1,152.0,207.0
1,2,152.0,207.0
2,3,709.0,564.0
3,4,85.0,551.0
4,5,89.0,57.0


In [0]:
ny_bldfront_blddepth.to_csv("Fill_BLDFRONT_BLDDEPTH.csv")